# CMS Ex-situ GIWAXS 2025
## Raw Data Processing & Exporting Notebook
Updated: 05.12.2025
- (2025C2): Incorporating pyFAI FiberIntegrator class - replacing pygix as the GI integrator. 
- (2023C2): In this notebook you output xr.DataSets stored as .zarr stores containing all your raw, remeshed (reciprocal space), and caked CMS GIWAXS data. Saving as a zarr automatically converts the array to a dask array.

In [ ]:
# # Outdated, this used to work to just overwrite existing PyHyper install in JupyterHub conda environment
# # If you need a custom PyHyper version install, you may need your own conda environment

# # Kernel updates if needed, remember to restart kernel after running this cell!:
# !pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

## Imports

In [ ]:
### Imports:
import gc, os, sys
import pathlib
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
from tqdm.auto import tqdm  # progress bar loader!

import PyHyperScattering as phs
print(f'Using PyHyperScattering Version: {phs.__version__}')

### Import PFFIGeneralIntegrator Local for Testing

In [ ]:
import importlib.util
from pathlib import Path
import warnings

# (optional) mute PyHyperScattering-wide UserWarnings
warnings.filterwarnings("ignore",
    ".*Unable to load optional dependency.*",
    category=UserWarning,
    module="PyHyperScattering"
)

# Path to your local file
file_path = Path("/Users/keithwhite/repos/PyHyperScattering/src/"
                 "PyHyperScattering/PFFIGeneralIntegrator.py")

spec = importlib.util.spec_from_file_location("local_pffig", str(file_path))
local_pffig = importlib.util.module_from_spec(spec)
spec.loader.exec_module(local_pffig)

# Grab the class
PFFIGeneralIntegrator = local_pffig.PFFIGeneralIntegrator

# Test it
print(PFFIGeneralIntegrator)


## Setup
- Define paths to data, calibrant, mask, .poni, etc.

### Define & Validate Paths
- Pathlib is preferred for its readability & checkability, it's also necessary for the loadSeries function later on.
- Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly.



In [ ]:
## Single Sample Example
propPath = pathlib.Path('/Users/keithwhite/repos/PyHyperScattering/example_data/cms-giwaxs-test')   ## Proposal path in NSLS-II directory.
dataPath = propPath.joinpath('/Users/keithwhite/repos/PyHyperScattering/example_data/cms-giwaxs-test/data')
rawPath = dataPath.joinpath('raw')  ## Build /raw/ data subfolder

## Multiple Samples 
# samplesPath = dataPath.joinpath('stitched')
maskponiPath = propPath.joinpath('maskponi')  # place for pyhyper-drawn masks and poni files

## Output Path
outPath = propPath.joinpath('output')

## Calibrant Info: Select poni & mask filepaths
poniFile = maskponiPath.joinpath('may23_poni4_nslsiimar23_12p7keV_CeO_KWPos_mask5_fit2.poni')
# maskFile = maskponiPath.joinpath('blank.json')
maskFile = maskponiPath.joinpath('may23_nslsiimar23_12p7keV_AgBh_KWPos_wSi_th0p3_mask_5.edf')
# calibPath = rawPath.joinpath('AgBH_cali_5m_12.7kev_x0.000_th0.000_10.00s_1307208_waxs.tiff')

## Colormap
cmap = plt.cm.turbo
cmap.set_bad('black')

### Look at how your metadata is formatted.
- You need to know the format of your metadata so you can define the parsing scheme in the next section.

#### Single File Set

In [ ]:
# Parameters
samp_delim = 'sam22*'
delim = '_'
expected_elements = None  # Set this to an integer (e.g. 5) to override

# Get all files matching the sample pattern
file_list = sorted(rawPath.glob(samp_delim))

if file_list:
    # Determine reference length
    ref_len = expected_elements if expected_elements is not None else len(file_list[0].name.split(delim))
    
    # Optionally print first file parts
    if expected_elements is None:
        ref_parts = file_list[0].name.split(delim)
        print("First file parts:", ref_parts)
    
    # Filter files by expected element count
    filtered_files = [f for f in file_list if len(f.name.split(delim)) == ref_len]
    
    print(f"Filtered files with {ref_len} elements:")
    print([f.name for f in filtered_files])
else:
    print("No matching files found.")


#### Multiple File Sets

In [ ]:
## Multiple Samples
# Checkout everything in position 1 (*pos1)
# [f.name for f in sorted(samplesPath.glob('*pos1*'))]

# [len(f.name.split('_')) for f in sorted(samplesPath.glob('*pos1*'))]

# [f.name for f in sorted(samplesPath.glob('*pos1*')) if len(f.name.split('_'))==9]

# fixed_rpm_set = [f for f in sorted(samplesPath.glob('*')) if len(f.name.split('_'))==9]
# variable_rpm_set = [f for f in sorted(samplesPath.glob('*')) if len(f.name.split('_'))==10]

# len(fixed_rpm_set)

### Metadata Scheme Definition
- 11-BM (CMS) parses metadata from filename strings, here is where you setup the parsing logic.

#### Single File Set

In [ ]:
# Parameters
delim = '_'
metadata_list = ['sample', 'material', 'filter', 
                 'concentration', 'flowrate', 'substrate', 
                 'solution_volume', 'runNumber', 'global_time', 
                 'xpos', 'incident_angle', 'exposure_time', 
                 'scan_id', 'scan_number', 'detector']

# Assume `file_set` is a list of Path objects (or strings) from globbing
# Example: file_set = sorted(rawPath.glob(samp_delim))
fi_fileset = sorted(rawPath.glob(samp_delim))  # Replace as appropriate

# Process the first file
if fi_fileset:
    first_file = fi_fileset[0]
    fields = first_file.name.split(delim)

    print(f"Metadata for file: {first_file.name}")
    if len(fields) != len(metadata_list):
        print(f"Mismatch: {len(fields)} fields in filename vs {len(metadata_list)} expected metadata labels")
    else:
        metadata_mapping = dict(zip(metadata_list, fields))
        for k, v in metadata_mapping.items():
            print(f"  {k}: {v}")
else:
    print("No files found matching the pattern.")


#### Multiple File Sets

In [ ]:
# # set ex situ metadata filename naming schemes:
# fixed_rpm_md_naming_scheme =    ['project', 'material', 'solvent', 'detector_pos', 'sample_pos', 
#                                  'incident_angle', 'exposure_time', 'scan_id', 'detector']
# variable_rpm_md_naming_scheme = ['project', 'material', 'solvent', 'rpm', 'detector_pos', 'sample_pos', 
#                                  'incident_angle', 'exposure_time', 'scan_id', 'detector']

# # A way to check our naming schemes to make sure they're right:
# delim = '_'
# file_sets =    [            fixed_rpm_set,               variable_rpm_set]
# file_schemes = [fixed_rpm_md_naming_scheme, variable_rpm_md_naming_scheme]

# for file_set, file_scheme in zip(file_sets, file_schemes):
#     first_filename = sorted(file_set)[0].name
#     print(f'\nFilename: {first_filename}')
#     first_filename_list = first_filename.split(delim)
#     for tup in zip(file_scheme, first_filename_list):
#         print(tup)

### Initialize Data Loader Objects
- This is where you load the data, if you didn't setup the metadata parsing correctly, this will fail.

#### Single File Set Example

In [ ]:
fi_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=metadata_list)

#### Multiple File Sets Example

In [ ]:
# # Initalize CMSGIWAXSLoader objects with the above naming schemes
# fixed_rpm_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=fixed_rpm_md_naming_scheme)
# variable_rpm_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=variable_rpm_md_naming_scheme)

## Data Processing
- Here is where the integration, masking, q-space conversion, etc. happens.
- Break this section up however makes sense for your data.

### PyFAI FiberIntegrator Example
- pyFAI 2025.03 updated their package to handle grazing incidence (GI) geometries. Due to this, we are moving away from using pygix for GI geometries, as pygix is deprecated and no longer supported.

#### Initialize Integrator

In [ ]:
import importlib.util
from pathlib import Path
import warnings

# (optional) mute PyHyperScattering-wide UserWarnings
warnings.filterwarnings("ignore",
    ".*Unable to load optional dependency.*",
    category=UserWarning,
    module="PyHyperScattering"
)

# Path to your local file
file_path = Path("/Users/keithwhite/repos/PyHyperScattering/src/"
                 "PyHyperScattering/PFFIGeneralIntegrator.py")

spec = importlib.util.spec_from_file_location("local_pffig", str(file_path))
local_pffig = importlib.util.module_from_spec(spec)
spec.loader.exec_module(local_pffig)

# Grab the class
PFFIGeneralIntegrator = local_pffig.PFFIGeneralIntegrator

# Test it
print(PFFIGeneralIntegrator)


In [ ]:
fi_integrator = PFFIGeneralIntegrator(
    geomethod='ponifile',
    ponifile=poniFile,
    # maskmethod='edf',
    # maskpath=maskFile,
    sample_orientation=4,
    incident_angle=0.3,
    tilt_angle=0,
    split_pixels=True)

# ## If/when this is integrated into the actual workflow and isn't local.
# fi_integrator = phs.integrate.PFFIGeneralIntegrator(geomethod = 'ponifile',
#                                       ponifile=poniFile,
#                                       sample_orientation=1,
#                                       incident_angle=0.3,
#                                       tilt_angle=0)

#### Initialize CMSGIWAXS Util Object
- Pass the ordered file set, loader object, and integrator object.

In [ ]:
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(fi_fileset), 
                                           fi_loader, 
                                           fi_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()


#### Plot to Validate Inputs

In [ ]:
%matplotlib tk

for DA in tqdm(recip_DS.data_vars.values()):
    plt.close('all')
    cmin, cmax = DA.quantile(0.01), DA.quantile(0.99)

    fig, ax = plt.subplots(figsize=(8, 4))
    DA.plot.imshow(
        ax=ax,
        cmap=cmap,
        norm=plt.Normalize(cmin, cmax),
        x='qip',
        y='qoop'
    )
    ax.set_aspect('equal')
    ax.set_title(
        f"{DA.material}, incident={DA.incident_angle}, scan={DA.scan_id}"
    )
    ax.set_ylim(bottom=0)

    plt.show()

#### Testing: Compare to PG Integrator

In [ ]:
import pygix
pg_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                    ponifile = poniFile,
                                    maskmethod='edf',
                                    maskpath=maskFile,
                                    output_space = 'recip')

util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(fi_fileset), fi_loader, pg_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()  # run function 
display(recip_DS)

for DA in tqdm(recip_DS.data_vars.values()):
    cmin, cmax = DA.quantile(0.01), DA.quantile(0.99)

    fig, ax = plt.subplots(figsize=(8, 4))
    DA.plot.imshow(
        ax=ax,
        cmap=cmap,
        norm=plt.Normalize(cmin, cmax),
        x='q_xy',
        y='q_z'
    )
    ax.set_aspect('equal')
    ax.set_title(
        f"{DA.material}, incident={DA.incident_angle}, scan={DA.scan_id}"
    )
    ax.set_ylim(bottom=0)
    plt.show()
    plt.close(fig)

### Using General Integrator (pyFAI)

#### Variable RPM file set

##### Intialize integrators

In [ ]:
variable_rpm_recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                                  ponifile = poniFile,
                                                                  output_space = 'recip')
variable_rpm_caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                                  ponifile = poniFile,
                                                                  output_space = 'caked')

##### Generate, check, save: recip Dataset

In [ ]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(variable_rpm_set), variable_rpm_loader, variable_rpm_recip_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()  # run function 
display(recip_DS)

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(list(recip_DS.data_vars.values())[::8]):   
#     cmin = DA.quantile(0.01)
#     cmax = DA.quantile(0.99)
    
#     ax = DA.sel(q_xy=slice(-1.1, 2.1), q_z=slice(-0.05, 2.4)).plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
#     ax.axes.set(aspect='equal', title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

##### generate, check, save: caked Dataset

In [ ]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(variable_rpm_set), variable_rpm_loader, variable_rpm_caked_integrator)
raw_DS, caked_DS = util.single_images_to_dataset()  # run function 
display(caked_DS)

In [ ]:
# Example of a quick plot check if desired here:
for DA in tqdm(list(caked_DS.data_vars.values())[::8]):   
    cmin = DA.quantile(0.01)
    cmax = DA.quantile(0.99)
    
    ax = DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
    ax.axes.set(title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
    plt.show()
    plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# caked_DS.to_zarr(savePath.joinpath(savename))

#### Fixed RPM file set

In [ ]:
# would be same code as above for another file set

## Not fully implemented

In [ ]:
def poni_centers(poniFile, pix_size=0.000172):
    """
    Returns poni center value and the corresponding pixel position. Default pixel size is 172 microns (Pilatus 1M)
    
    Inputs: poniFile as pathlib path object to the poni file
    Outputs: ((poni1, y_center), (poni2, x_center))
    """
    
    with poniFile.open('r') as f:
        lines = list(f.readlines())
    poni1_str = lines[6]
    poni2_str = lines[7]

    poni1 = float(poni1_str.split(' ')[1])
    poni2 = float(poni2_str.split(' ')[1])

    y_center = poni1 / pix_size
    x_center = poni2 / pix_size
        
    return ((poni1, y_center), (poni2, x_center))

poni_y, poni_x = poni_centers(poniFile)
display(poni_y)
display(poni_x)

### Yoneda check:
This can be used as a way to verify / refine your correct beam center y position. The yoneda peak should always appear at a q value corresponding to your incident angle plus your film's critical angle:

In [ ]:
def yoneda_qz(wavelength, alpha_crit, alpha_incidents):
    """Calculate the yoneda qz values given the wavelength, critical angle, and incident angle (in degrees)"""
    qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad((alpha_incidents + alpha_crit)/2)))
    qz_inv_angstroms = qz_inv_meters / 1e10
    return qz_inv_angstroms


wavelength = 9.762535309700809e-11  # 12.7 keV
alpha_crit = 0.11  # organic film critical angle
alpha_incidents = np.array([0.08, 0.1, 0.12, 0.15])  # incident angle(s)

yoneda_angles = alpha_incidents + alpha_crit

yoneda_qz(wavelength, alpha_crit, alpha_incidents)  # expected yoneda qz positions

In [ ]:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

In [ ]:
# 2D reciprocal space cartesian plots
qxy_min = -1.1
qxy_max = 2.1
qz_min = -0.01
qz_max = 2.2

selected_attrs_dict = {'material': ['PM6'], 'solvent': ['CBCN']}
# selected_attrs_dict = {}

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    
    real_min = float(sliced_DA.compute().quantile(0.05))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.997))   
    
    # Plot
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    # ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent} {DA.rpm}, {float(DA.incident_angle[2:])}° Incidence',
    #             xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()
    plt.close('all')

In [ ]:
# Yoneda peak linecut check
qxy_min = 0.22
qxy_max = 2
qz_min = -0.02
qz_max = 0.06

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    qz_integrated_DA = sliced_DA.sum('q_xy')
    
    # Plot
    qz_integrated_DA.plot.line(label=DA.incident_angle)
    
plt.legend()
plt.grid(visible=True, which='major', axis='x')
plt.show()

In [ ]:
chi_min = 60
chi_max = None

selected_DAs = select_attrs(fixed_caked_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max))
    
    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.05))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.999))  
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, 10), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(title=f'Polar Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(outPath.joinpath('PM6-Y6set_waxs', f'polar-2D_{DA.sample_id}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')